# 🕸️ Tutorial 4: Building Knowledge Graphs

**Learn to build knowledge graphs from research papers using AI.**

## What You'll Learn:
- Extract entities from papers using AI
- Build knowledge graphs automatically
- Visualize research connections
- Query graphs for insights

**Time:** 15 minutes | **Level:** Beginner

## Step 1: Setup

In [1]:
# Import what we need
import sys
import os

# Add parent directory to path
if os.path.basename(os.getcwd()) == 'tutorial':
    sys.path.insert(0, '..')
else:
    sys.path.insert(0, '.')

# Enable widgets for visualization
try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
    print("📱 Google Colab widget support enabled")
except:
    pass

# Import the GraphRAG system
from src.langchain_graph_rag import LangChainGraphRAG

print("✅ Setup complete!")

✅ Setup complete!


## Step 2: Create GraphRAG System

In [2]:
# Create knowledge graph system
graph_rag = LangChainGraphRAG(
    llm_model="llama3.1:8b",
    embedding_model="nomic-embed-text"
)

print("🕸️ Knowledge graph system ready!")
print(f"📊 Current papers: {len(graph_rag.get_all_papers())}")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:src.langchain_graph_rag:🕸️ LangChain GraphRAG initialized


🕸️ Knowledge graph system ready!
📊 Current papers: 2


## Step 3: Add Your First Paper

In [ ]:
# Load a real research paper instead of sample text
from src import SimplePaperRAG

print("📄 Loading real research paper...")
print("⏳ This will extract much more comprehensive entities...")

# Load actual PDF paper
paper_path = "../examples/d4sc03921a.pdf"  # Main research paper
rag_system = SimplePaperRAG()
paper_info = rag_system.load_paper(paper_path)

# Get the full paper content and metadata
paper_content = rag_system.paper_data['content']
paper_title = rag_system.paper_data['title']

print(f"✅ Paper loaded successfully!")
print(f"📰 Title: {paper_title[:100]}...")
print(f"📊 Content length: {len(paper_content):,} characters")
print(f"📄 This is a full research paper with comprehensive content!")

# Add paper to knowledge graph
result = graph_rag.extract_entities_and_relationships(
    paper_content=paper_content,
    paper_title=paper_title,
    paper_id="paper_1"
)

print(f"\n✅ Real paper added to knowledge graph!")
print(f"📝 Documents created: {result['documents_added']}")
print(f"🏷️ Entities extracted: {sum(len(entities) for entities in result['entities'].values())}")
print(f"📊 Graph nodes: {result.get('graph_stats', {}).get('nodes', 'N/A')}")
print(f"🕸️ Graph edges: {result.get('graph_stats', {}).get('edges', 'N/A')}")

## Step 4: See What AI Found

In [4]:
# Show extracted entities
entities = result['entities']

print("🤖 AI Found These Entities:")
print("=" * 30)

for category, entity_list in entities.items():
    if entity_list:
        print(f"\n📋 {category.upper()}:")
        for entity in entity_list:
            print(f"   • {entity}")

🤖 AI Found These Entities:

📋 AUTHORS:
   • Dr. Sarah Chen
   • Prof. Michael Torres

📋 INSTITUTIONS:
   • MIT
   • Stanford

📋 DATASETS:
   • PubChem
   • ChEMBL


## Step 5: Add Another Paper

In [ ]:
# Load a second real research paper for comparison
print("📄 Loading second research paper...")

# Load second PDF paper
paper_path_2 = "../examples/d3dd00113j.pdf"  # Second research paper
rag_system_2 = SimplePaperRAG()
paper_info_2 = rag_system_2.load_paper(paper_path_2)

# Get the content and metadata
paper_content_2 = rag_system_2.paper_data['content']
paper_title_2 = rag_system_2.paper_data['title']

print(f"✅ Second paper loaded!")
print(f"📰 Title: {paper_title_2[:100]}...")
print(f"📊 Content length: {len(paper_content_2):,} characters")

# Add second paper to knowledge graph
result_2 = graph_rag.extract_entities_and_relationships(
    paper_content=paper_content_2,
    paper_title=paper_title_2, 
    paper_id="paper_2"
)

print(f"\n✅ Second real paper added!")
print(f"📚 Total papers in graph: {result_2.get('total_papers', 'N/A')}")
print(f"🏷️ New entities extracted: {sum(len(entities) for entities in result_2['entities'].values())}")

# Show total system stats
total_papers = result_2.get('total_papers', 2)
print(f"\n📈 Knowledge Graph now contains:")
print(f"   📄 Papers: {total_papers}")
print(f"   📊 Rich entities from real research content")
print(f"   🔬 Comprehensive academic vocabulary")
print(f"   🕸️ Complex relationship networks")

## Step 6: Find Connections

In [6]:
# Find papers connected by shared authors
connections = graph_rag.find_related_papers("paper_1", "authors")

print("🔗 Paper Connections Found:")
print("=" * 30)

if connections['related_papers']:
    for paper_id, info in connections['related_papers'].items():
        print(f"\n📄 {info['paper_title']}")
        print(f"   🔗 Shared authors: {', '.join(info['shared_entities'])}")
else:
    print("No connections found")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


🔗 Paper Connections Found:

📄 Neural Networks in Chemical Research
   🔗 Shared authors: Prof. Michael Torres


## Step 7: Query the Knowledge Graph

In [7]:
# Ask questions about your research
query = "machine learning and chemistry"

results = graph_rag.query_graph(query)

print(f"🔍 Query: '{query}'")
print("=" * 40)
print(f"📊 Found {results['papers_found']} relevant papers")

for paper_id, paper_data in results['papers'].items():
    print(f"\n📄 {paper_data['paper_title']}")
    print(f"   💬 Relevant sections: {len(paper_data['chunks'])}")
    
    # Show snippet
    if paper_data['chunks']:
        snippet = paper_data['chunks'][0][:100]
        print(f"   📝 {snippet}...")

/Users/aimiegarces/Agents/tutorial/../src/langchain_graph_rag.py:80: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_docs = self.retriever.get_relevant_documents(query)
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.

🔍 Query: 'machine learning and chemistry'
📊 Found 2 relevant papers

📄 Machine Learning for Drug Discovery
   💬 Relevant sections: 1
   📝 Machine Learning for Drug Discovery
Authors: Dr. Sarah Chen (MIT), Prof. Michael Torres (Stanford)

...

📄 Neural Networks in Chemical Research
   💬 Relevant sections: 1
   📝 Neural Networks in Chemical Research
Authors: Prof. Michael Torres (Stanford), Dr. Elena Rodriguez (...


## Step 8: Visualize Your Knowledge Graph

In [8]:
# Visualize Your Knowledge Graph
from src.notebook_visualization import show_knowledge_graph

print("🎨 Creating knowledge graph visualization...")

# Display interactive graph with professional features
result = show_knowledge_graph(graph_rag)

if result:
    print("\n💡 If you see a graph above, you can:")
    print("   • Drag nodes to rearrange")
    print("   • Use sidebar to explore properties") 
    print("   • Search for specific entities")
    print("   • Zoom and pan to navigate")
else:
    print("\n⚠️ Visualization had issues - but your knowledge graph is working!")
    
    # Show what we built anyway
    summary = graph_rag.get_graph_summary()
    print(f"\n📊 Your Knowledge Graph:")
    print(f"   📄 Papers: {summary['total_papers']}")
    print(f"   📝 Documents: {summary['total_documents']}")
    print(f"   🏷️ Entities: {sum(summary['unique_entities'].values())}")
    print("\n🎉 Knowledge graph built successfully!")

🎨 Creating knowledge graph visualization...


GraphWidget(layout=Layout(height='500px', width='100%'))

✅ Interactive yFiles graph displayed above!
🎯 Use the sidebar to explore nodes and relationships
🔍 Try the search function to find specific entities

💡 If you see a graph above, you can:
   • Drag nodes to rearrange
   • Use sidebar to explore properties
   • Search for specific entities
   • Zoom and pan to navigate


## Step 9: Explore Your Graph

In [9]:
# Get overview of your knowledge graph
summary = graph_rag.get_graph_summary()

print("📊 Knowledge Graph Summary:")
print("=" * 30)
print(f"📄 Papers: {summary['total_papers']}")
print(f"📝 Document chunks: {summary['total_documents']}")

print("\n🏷️ Unique Entities:")
for entity_type, count in summary['unique_entities'].items():
    if count > 0:
        print(f"   • {entity_type.title()}: {count}")

print("\n🎉 You built a knowledge graph!")

📊 Knowledge Graph Summary:
📄 Papers: 2
📝 Document chunks: 28

🏷️ Unique Entities:
   • Authors: 3
   • Institutions: 3
   • Datasets: 2

🎉 You built a knowledge graph!


## Try It Yourself!

In [10]:
# Add your own paper content here!
your_paper = """
Replace this with content from your research area:
- Title and authors
- Abstract or summary
- Key methods and findings
- Datasets used
"""

# Uncomment and modify to add your paper:
# result = graph_rag.extract_entities_and_relationships(
#     paper_content=your_paper,
#     paper_title="Your Paper Title",
#     paper_id="your_paper"
# )
# print("✅ Your paper added to the knowledge graph!")

print("💡 Add your own research content above to expand the graph!")

💡 Add your own research content above to expand the graph!


## 🎓 What You Learned

**Congratulations!** You built an AI-powered knowledge graph that:

✅ **Extracts entities** from research papers automatically  
✅ **Finds connections** between different papers  
✅ **Answers questions** about your research  
✅ **Visualizes relationships** in an interactive graph  

### 🚀 Next Steps:
- Add real papers from your research area
- Try different query types
- Explore the interactive visualization
- Scale up to larger paper collections

### 🔗 Real-world Applications:
- **Literature reviews** - Find research gaps
- **Collaboration mapping** - Discover research networks  
- **Citation analysis** - Track research influence
- **Knowledge discovery** - Uncover hidden connections